In [1]:
import pandas as pd

### 1. 날씨 데이터 전처리

In [2]:
weather = pd.read_csv("./weather.csv")
weather.head()

,지점,지점명,일시,평균기온,일강수량,평균 풍속,평균 상대습도,일 최심적설
0,108,서울,2018-10-01,15.4,0.0,2.9,55.1,0.0
1,108,서울,2018-10-02,15.9,0.0,1.6,61.4,0.0
2,108,서울,2018-10-03,17.3,0.0,1.3,58.0,0.0
3,108,서울,2018-10-04,19.3,0.0,1.3,56.5,0.0
4,108,서울,2018-10-05,16.9,36.5,1.7,84.1,0.0


In [3]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지점       365 non-null    int64  
 1   지점명      365 non-null    object 
 2   일시       365 non-null    object 
 3   평균기온     365 non-null    float64
 4   일강수량     365 non-null    float64
 5   평균 풍속    365 non-null    float64
 6   평균 상대습도  365 non-null    float64
 7   일 최심적설   365 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 22.9+ KB


In [4]:
# 쓸데없는 컬럼 제거
weather.drop(columns=["지점", "지점명"], inplace=True)

In [5]:
# 컬럼명 정리
weather.rename(columns={"일시" : "일자",
                        "평균기온" : "기온",
                       "일강수량" : "강수량",
                       "평균 풍속" : "풍속",
                       "평균 상대습도" : "습도",
                       "일 최심적설" : "적설량"}, inplace=True)

In [6]:
# 배달데이터와 일자 형식 맞추기
weather["일자"] = weather["일자"].str.replace("-", "")

In [7]:
weather

,일자,기온,강수량,풍속,습도,적설량
0,20181001,15.4,0.0,2.9,55.1,0.0
1,20181002,15.9,0.0,1.6,61.4,0.0
2,20181003,17.3,0.0,1.3,58.0,0.0
3,20181004,19.3,0.0,1.3,56.5,0.0
4,20181005,16.9,36.5,1.7,84.1,0.0
...,...,...,...,...,...,...
360,20190926,22.9,0.0,1.9,59.3,0.0
361,20190927,22.9,0.0,1.9,57.5,0.0
362,20190928,23.7,0.0,1.7,58.9,0.0
363,20190929,22.6,0.0,1.7,60.6,0.0


### 2. 배달 데이터 전처리

In [8]:
# 강남구 데이터
delivery = pd.read_csv("./delivery.csv")
delivery.head()

,일자,요일,시간대,시도,시군구,읍면동,통화건수
0,20181001,월,0,서울특별시,강남구,논현동,5
1,20181001,월,0,서울특별시,강남구,세곡동,5
2,20181001,월,0,서울특별시,강남구,삼성동,7
3,20181001,월,0,서울특별시,강남구,역삼동,5
4,20181001,월,0,서울특별시,강서구,등촌동,5


In [9]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383050 entries, 0 to 383049
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   일자      383050 non-null  int64 
 1   요일      383050 non-null  object
 2   시간대     383050 non-null  int64 
 3   시도      383050 non-null  object
 4   시군구     383050 non-null  object
 5   읍면동     383050 non-null  object
 6   통화건수    383050 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 20.5+ MB


In [10]:
# 날짜별로 groupby
delivery = delivery.groupby(["일자", "요일"]).sum().drop(columns="시간대").reset_index()

In [11]:
# '일자' 컬럼 데이터형 일치시키기
delivery["일자"] = delivery["일자"].astype('str')
delivery

,일자,요일,통화건수
0,20181001,월,9681
1,20181002,화,13692
2,20181003,수,15403
3,20181004,목,10091
4,20181005,금,17680
...,...,...,...
360,20190926,목,8908
361,20190927,금,12561
362,20190928,토,13828
363,20190929,일,12188


In [12]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      365 non-null    object
 1   요일      365 non-null    object
 2   통화건수    365 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 8.7+ KB


### 3. 공휴일 데이터 전처리

In [118]:
holiday = pd.read_csv('./national_holiday.csv', index_col=0)

In [135]:
holiday.rename(columns={'date':'일자', 'datename':'공휴일'}, inplace=True)
holiday['일자'] = holiday['일자'].astype(str)
holiday['공휴일'] = 1
holiday.head()

,일자,공휴일
15,20181003,1
16,20181009,1
17,20181225,1
18,20190101,1
19,20190204,1


### 4. 날씨+공휴일+배달 데이터 합치기

In [120]:
# 날씨+배달
weather_delivery= pd.merge(weather, delivery, on="일자")

In [136]:
# 날씨+공휴일+배달
df = pd.merge(weather_delivery, holiday, how='outer')
df = df[["일자", "요일", "공휴일", "기온", "강수량", "풍속", "습도", "적설량", "통화건수"]]

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 0 to 364
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      365 non-null    object 
 1   요일      365 non-null    object 
 2   공휴일     16 non-null     float64
 3   기온      365 non-null    float64
 4   강수량     365 non-null    float64
 5   풍속      365 non-null    float64
 6   습도      365 non-null    float64
 7   적설량     365 non-null    float64
 8   통화건수    365 non-null    int64  
dtypes: float64(6), int64(1), object(2)
memory usage: 28.5+ KB


In [138]:
df['공휴일'].fillna(0, inplace=True)

In [139]:
df.head()

,일자,요일,공휴일,기온,강수량,풍속,습도,적설량,통화건수
0,20181001,월,0.0,15.4,0.0,2.9,55.1,0.0,9681
1,20181002,화,0.0,15.9,0.0,1.6,61.4,0.0,13692
2,20181003,수,1.0,17.3,0.0,1.3,58.0,0.0,15403
3,20181004,목,0.0,19.3,0.0,1.3,56.5,0.0,10091
4,20181005,금,0.0,16.9,36.5,1.7,84.1,0.0,17680


In [140]:
df.to_csv("df.csv")

### 5. 카테고리변수 인코딩

In [142]:
# 요일변수 원핫인코딩
df_dummy = pd.concat([df, pd.get_dummies(df['요일'])], axis=1)
df_dummy.head()

,일자,요일,공휴일,기온,강수량,풍속,습도,적설량,통화건수,금,목,수,월,일,토,화
0,20181001,월,0.0,15.4,0.0,2.9,55.1,0.0,9681,0,0,0,1,0,0,0
1,20181002,화,0.0,15.9,0.0,1.6,61.4,0.0,13692,0,0,0,0,0,0,1
2,20181003,수,1.0,17.3,0.0,1.3,58.0,0.0,15403,0,0,1,0,0,0,0
3,20181004,목,0.0,19.3,0.0,1.3,56.5,0.0,10091,0,1,0,0,0,0,0
4,20181005,금,0.0,16.9,36.5,1.7,84.1,0.0,17680,1,0,0,0,0,0,0


In [143]:
df_dummy = df_dummy[['일자', '요일', '공휴일', '월', '화', '수', '목', '금', '토', '일', '기온', '강수량', '풍속', '습도', '적설량', '통화건수']]
df_dummy.tail()

,일자,요일,공휴일,월,화,수,목,금,토,일,기온,강수량,풍속,습도,적설량,통화건수
360,20190926,목,0.0,0,0,0,1,0,0,0,22.9,0.0,1.9,59.3,0.0,8908
361,20190927,금,0.0,0,0,0,0,1,0,0,22.9,0.0,1.9,57.5,0.0,12561
362,20190928,토,0.0,0,0,0,0,0,1,0,23.7,0.0,1.7,58.9,0.0,13828
363,20190929,일,0.0,0,0,0,0,0,0,1,22.6,0.0,1.7,60.6,0.0,12188
364,20190930,월,0.0,1,0,0,0,0,0,0,22.4,0.0,1.6,61.0,0.0,7850


In [144]:
df_dummy.to_csv('df_dummy.csv')